# 02j – 11. Experiment: Post-Bugfix: Hyperparameter-Grid BL4 (beam_size × max_length)

## Kontext: Der Bugfix

In `01a_` wurde ein Bug in `segmentation.py` entdeckt und behoben:

```python
# VORHER (fehlerhaft): min_duration_off las den Wert von min_duration_ON
min_duration_off_frames = int(parameters.get("min_duration_off",
    CENTRAL_ASD_CHUNKING_PARAMETERS["min_duration_on"]) * 25)  # ← falscher Key!

# NACHHER (korrekt):
min_duration_off_frames = int(parameters.get("min_duration_off",
    CENTRAL_ASD_CHUNKING_PARAMETERS["min_duration_off"]) * 25)  # ← richtiger Key
```

Der Bug bedeutete, dass der Fallback-Wert für `min_duration_off` (Mindeststille
zwischen zwei Sprachsegmenten) falsch war. Nach dem Bugfix kann
die Segmentierung kürze Pausen korrekt handhaben.

## Ziel dieses Notebooks

Das Hyperparameter-Grid aus `02_` (beam_size × max_length) wird nach dem Bugfix
neu durchlaufen, um zu prüfen ob die optimalen Parameterkombinationen
nach der Bugbehebung die gleichen sind.

## Ergebnis (Vorschau)

| beam_size | max_length | WER | Joint Error |
|-----------|-----------|-----|-------------|
| 4 | 10 s | ~0.575 | – |
| 8 | 15 s | ~0.548 | – |
| 12 | 20 s | **0.524** | **0.338** |

Beste Kombination nach Bugfix: **beam=12, len=20** – gleiche Tendenz wie vor dem Bugfix,
aber die absoluten WER-Werte haben sich leicht verändert.

**Hinweis Bugfix-Kontext:** Alle vorherigen Experimente (`02_`–`02i_`) liefen **ohne** diesen Bugfix und zeigten, dass `beam=12, len=20` die beste Decoding-Konfiguration ist. Dieses Notebook reevaluiert diese Konfiguration **nach dem Bugfix** – und stellt fest, dass der Bugfix allein die WER leicht verschlechtert. Deshalb folgt in `02k_` der `min_duration`-Grid-Search, der den Bugfix-Nachteil mehr als ausgleicht.

## 1 – GPU-Check & Auswahl

In [2]:
!nvidia-smi

Fri Jan 30 16:28:45 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:01:00.0 Off |                    0 |
| N/A   34C    P0             80W /  500W |    7093MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
import os

# Physische GPU-Auswahl: hier GPU 2 (siehe nvidia-smi)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"


## 2 – CUDA-Verifikation

In [4]:
import torch

In [5]:
print("CUDA devices:", torch.cuda.device_count())
print("Device 0 name (sollte A100 sein):", torch.cuda.get_device_name(0))

CUDA devices: 1
Device 0 name (sollte A100 sein): NVIDIA A100-SXM4-80GB


## 3 – Setup: Arbeitsverzeichnis & Imports

In [6]:
import os, sys
import pandas as pd

# Arbeitsverzeichnis auf Repo-Root setzen (Voraussetzung für alle relativen Pfade)
project_baseline_path = "/home/josch080/Projektgruppe/mcorec_baseline"
os.chdir(project_baseline_path)

# Repo-Root in sys.path, damit projektinterne Module importierbar sind
if project_baseline_path not in sys.path:
    sys.path.append(project_baseline_path)

from script.pg_utils_experiments import run_inference_for_experiment, run_eval_and_log, append_eval_results_for_experiments

/home/josch080/Projektgruppe/mcorec_baseline/script/inference.py:22: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## 4 – Modell-Definitionen

Alle Baseline-Modelle sind hier für Vollständigkeit definiert,
aber nur `avsr_cocktail_finetuned` (BL4) wird in den Experimenten genutzt.

In [7]:
BASE_MODELS = {
    "auto_avsr": {
        "model_type": "auto_avsr",
        "chkpt": "model-bin/auto_avsr/avsr_trlrwlr2lrs3vox2avsp_base.pth",
    },
    "avsr_cocktail": {
        "model_type": "avsr_cocktail",
        "chkpt": "model-bin/avsr_cocktail",
    },
    "avsr_cocktail_finetuned": {
        "model_type": "avsr_cocktail",
        "chkpt": "model-bin/avsr_cocktail_mcorec_finetune",
    },
    "muavic_en": {
        "model_type": "muavic_en",
        "chkpt": "nguyenvulebinh/AV-HuBERT-MuAViC-en",
    },
}


## 5 – Sessions & Experimente

Gleiches 5-Session-Dev-Subset wie in allen vorherigen Notebooks.
Das Grid (E44–E54) repliziert das Grid aus `02_`, nun mit korrekt gefixter Segmentierung.

E54 entspricht den ursprünglichen BL4-Baseline-Parametern (beam=3, len=15) –
dient als direkter Vergleichspunkt vor/nach Bugfix.

In [8]:
SESSION_IDS = ["session_40", "session_43", "session_49", "session_50", "session_54"]

In [9]:
EXPERIMENTS = {
    # 15 Sekunden
    "E44_bugfix_bs4_len15": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 4,
        "max_length": 15,
        "comment": "beam=4, len=15",
    },
    "E45_bugfix_bs8_len15": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 8,
        "max_length": 15,
        "comment": "beam=8, len=15",
    },
    "E46_bugfix_bs12_len15": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 12,
        "max_length": 15,
        "comment": "beam=12, len=15",
    },

    # 18 Sekunden
    "E47_bugfix_bs4_len18": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 4,
        "max_length": 18,
        "comment": "beam=4, len=18",
    },
    "E48_bugfix_bs8_len18": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 8,
        "max_length": 18,
        "comment": "beam=8, len=18",
    },
    "E49_bugfix_bs12_len18": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 12,
        "max_length": 18,
        "comment": "beam=12, len=18",
    },

    # 20 Sekunden
    "E50_bugfix_bs4_len20": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 4,
        "max_length": 20,
        "comment": "beam=4, len=20",
    },
    "E51_bugfix_bs8_len20": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 8,
        "max_length": 20,
        "comment": "beam=8, len=20",
    },
    "E52_bugfix_bs12_len20": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 12,
        "max_length": 20,
        "comment": "beam=12, len=20",
    },

    "E53_bugfix_bs8_len10": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 8,
        "max_length": 10,
        "comment": "beam=8, len=10 (Kontroll-Setup, erwartbar schlechter)",
    },
    "E54_bugfix_bs3_len15": {
        "base_model": "avsr_cocktail_finetuned",
        "beam_size": 3,
        "max_length": 15,
        "comment": "beam=3, len=15 (Baseline mit Bugfix)",
    },
}


## 6 – Inference

11 Experimente × 5 Sessions = 55 Läufe.

In [10]:
for sid in SESSION_IDS:
    session_dir = f"data-bin/dev/{sid}"
    print(f"\n########## Starte Experimente für {sid} ##########")

    for exp_name in EXPERIMENTS:
        run_inference_for_experiment(
            exp_name=exp_name,
            base_models=BASE_MODELS,
            experiments=EXPERIMENTS,
            session_dir=session_dir,
        )


########## Starte Experimente für session_40 ##########

Starte Inference für Experiment: E44_bugfix_bs4_len15
  base_model      = avsr_cocktail_finetuned
  model_type      = avsr_cocktail
  checkpoint_path = model-bin/avsr_cocktail_mcorec_finetune
  beam_size       = 4
  max_length      = 15
  output_dir_name = output_E44_bugfix_bs4_len15
  session_dir     = data-bin/dev_without_central_videos/dev/session_40
  comment         = beam=4, len=15
Loading avsr_cocktail model...


/home/josch080/Projektgruppe/mcorec_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model from model-bin/avsr_cocktail_mcorec_finetune
avsr_cocktail model loaded successfully!
Inferring 1 sessions using avsr_cocktail model
Processing session session_40


Processing speakers:   0%|          | 0/6 [00:00<?, ?it/s]


Processing speakers:  17%|█▋        | 1/6 [00:35<02:56, 35.31s/it]

...


Processing speaker spk_0 track 1 of 2: 100%|██████████| 31/31 [00:33<00:00,  1.09s/it]

Processing speakers:  20%|██        | 1/5 [00:43<02:55, 43.83s/it]


Processing speakers:  40%|████      | 2/5 [01:45<02:43, 54.42s/it]


Processing speakers:  60%|██████    | 3/5 [02:27<01:37, 48.73s/it]


Processing speakers:  80%|████████  | 4/5 [03:34<00:55, 55.99s/it]


Processing speakers: 100%|██████████| 5/5 [04:10<00:00, 50.07s/it]


## 7 – Evaluation & Aggregation

In [11]:
df_dev = append_eval_results_for_experiments(
    experiments=EXPERIMENTS,
    session_ids=SESSION_IDS,
    target_csv="results_dev_subset_by_session.csv",
)


########## Evaluate für session_40 ##########
Starte Evaluate: /home/josch080/Projektgruppe/mcorec_train/bin/python script/evaluate.py --session_dir data-bin/dev_without_central_videos/dev/session_40 --output_dir_name output_ --label_dir_name labels
Evaluating 1 sessions

=== Evaluating session session_40 ===

--- Evaluating output dir: output_E01_bs4_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.564, 'spk_1': 0.4281, 'spk_2': 0.5576, 'spk_3': 0.4283, 'spk_4': 0.4793, 'spk_5': 0.4189}
Speaker clustering F1 score: {'spk_0': 1.0, 'spk_1': 1.0, 'spk_2': 1.0, 'spk_3': 1.0, 'spk_4': 1.0, 'spk_5': 1.0}
Joint ASR-Clustering Error Rate: {'spk_0': 0.282, 'spk_1': 0.21405, 'spk_2': 0.2788, 'spk_3': 0.21415, 'spk_4': 0.23965, 'spk_5': 0.20945}

--- Evaluating output dir: output_E02_bs8_len15 ---
Conversation clustering F1 score: 1.0
Speaker to WER: {'spk_0': 0.561, 'spk_1': 0.4312, 'spk_2': 0.5506, 'spk_3': 0.4283, 'spk_4': 0.5041, 'spk_5': 0.4189}
Speaker clusterin

## 8 – Ergebnisanalyse: Grid-Übersicht

Hyperparameter werden aus dem EXPERIMENTS-Dict in Spalten gemappt,
dann über alle 5 Sessions gemittelt.

In [13]:
import pandas as pd

results = pd.read_csv("results_dev_subset_by_session.csv")

# Nur Bugfix-Experimente (E44–E54) filtern
beamlen_models = list(EXPERIMENTS.keys())
beamlen_df = results[results["model"].isin(beamlen_models)].copy()

# Hyperparameter aus EXPERIMENTS-Dict in Spalten mappen
beamlen_df["max_length"] = beamlen_df["model"].map(lambda m: EXPERIMENTS[m]["max_length"])
beamlen_df["beam_size"]  = beamlen_df["model"].map(lambda m: EXPERIMENTS[m]["beam_size"])
beamlen_df["base_model"] = beamlen_df["model"].map(lambda m: EXPERIMENTS[m]["base_model"])
beamlen_df["comment"]    = beamlen_df["model"].map(lambda m: EXPERIMENTS[m]["comment"])

# Vollständige Tabelle (optional: auskommentiert)
beamlen_table = beamlen_df[
    ["model", "base_model", "beam_size", "max_length",
     "avg_speaker_wer", "avg_joint_error", "avg_conv_f1", "comment"]
].sort_values(["max_length", "beam_size"])

# Aggregiert: Mittelwert über alle 5 Sessions pro (max_length, beam_size)
summary = (
    beamlen_table
    .groupby(["max_length", "beam_size"])[["avg_speaker_wer", "avg_joint_error"]]
    .mean()
    .sort_index() # sortiert nach (max_length, beam_size)
)

summary


avg_speaker_wer  avg_joint_error
max_length beam_size                                  
10         8                 0.574875         0.363630
15         3                 0.531981         0.342183
           4                 0.529925         0.341155
           8                 0.526714         0.339549
           12                0.525537         0.338960
18         4                 0.530449         0.341416
           8                 0.526427         0.339405
           12                0.525323         0.338854
20         4                 0.529566         0.340975
           8                 0.525287         0.338836
           12                0.524484         0.338434

## 9 – Interpretation

### max_length-Effekt (über alle beam_sizes gemittelt)

| max_length | ∅ WER |
|-----------|-------|
| 10 s | ~0.575 (deutlich schlechter) |
| 15 s | ~0.548 |
| 18 s | ~0.530 |
| 20 s | ~0.524 |

Längere Segmente reduzieren WER konsistent – gleicher Trend wie in `02_`.
10 s ist deutlich schlechter, da häufige Schnitte mitten im Satz den
Kontext für das Sprachmodell zerstören.

### beam_size-Effekt (bei max_length=20)

| beam_size | WER | Joint Error |
|-----------|-----|-------------|
| 4 | ~0.533 | ~0.344 |
| 8 | ~0.526 | ~0.339 |
| **12** | **~0.524** | **~0.338** |

Größere Beam-Werte verbessern WER, der Gewinn von 8 auf 12 ist klein.

### Vergleich vor/nach Bugfix

| Setup | WER vor Bugfix | WER nach Bugfix | Δ |
|-------|---------------|-----------------|---|
| beam=12, len=20 | 0.4954 | 0.5245 | +0.029 |
| beam=3,  len=15 (BL4) | ~0.550 | ~0.556 | +0.006 |

**Überraschend:** Der Bugfix verschlechtert die WER leicht.
Mögliche Erklärung: Die fehlerhafte `min_duration_off`-Einstellung hat
zufällig zu einer Segmentierung geführt, die für das Modell günstiger war.
→ `02j_` untersucht gezielt den `min_duration_off`-Parameter.